<a href="https://colab.research.google.com/github/KNGLJordan/CDMO-project/blob/main/src/SAT/sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install python3-z3

[sudo] password for lucazini03: 


In [1]:
from z3 import *
from itertools import combinations

In [2]:
#the man the myth the legend: angelo quarta
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

In [7]:
from itertools import combinations
from z3 import *
import time

import numpy as np

# Get number of teams from user
n = int(input("Enter even number of teams: "))

# Start timer
start_time = time.time()

# === Build schedule ===
if n % 2 != 0:
    raise ValueError("Number of teams must be even.")

weeks = n - 1
periods = n // 2
teams = list(range(n))


home = np.empty((n, weeks, periods), dtype=object)
away = np.empty((n, weeks, periods), dtype=object)
for t in teams:
    for w in range(weeks):
        for p in range(periods):
            home[t, w, p] = Bool(f"H_{t}_{w}_{p}")
            away[t, w, p] = Bool(f"A_{t}_{w}_{p}")

s = Solver()

# 0. Each slot has exactly one home and one away
for w in range(weeks):
    for p in range(periods):
        s.add(exactly_one_he([home[t, w, p] for t in teams], name=f"slot_home_{w}_{p}"))
        s.add(exactly_one_he([away[t, w, p] for t in teams], name=f"slot_away_{w}_{p}"))

# 1. Every team plays exactly once per week
for t in teams:
    for w in range(weeks):
        vars_in_week = [home[t, w, p] for p in range(periods)] + \
                       [away[t, w, p] for p in range(periods)]
        s.add(exactly_one_he(vars_in_week, name=f"team_plays_{t}_{w}"))

# 2. Every pair meets exactly once
for i, j in combinations(teams, 2):
    match_slots = []
    for w in range(weeks):
        for p in range(periods):
            match_slots.append(
                Or(And(home[i, w, p], away[j, w, p]),
                   And(home[j, w, p], away[i, w, p]))
            )
    s.add(exactly_one_he(match_slots, name=f"pair_meets_{i}_{j}"))

# 3. At most 2 games per team per period
for t in teams:
    for p in range(periods):
        games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
        s.add(at_most_k_seq(games, 2, name=f"team_{t}_period_{p}"))

ideal = weeks // 2
for t in teams:
    home_games = [home[t, w, p] for w in range(weeks) for p in range(periods)]
    s.add(at_least_k_seq(home_games, ideal, f"home_min_{t}"))
    s.add(at_most_k_seq(home_games, ideal+1, f"home_max_{t}"))


# Solve
if s.check() != sat:
    sched = None
else:
    m = s.model()

    # Extract schedule
    sched = [[None]*weeks for _ in range(periods)]
    for w in range(weeks):
        for p in range(periods):
            for i in teams:
                if m.evaluate(home[i, w, p]):
                    for j in teams:
                        if m.evaluate(away[j, w, p]):
                            sched[p][w] = (i+1, j+1)

# End timer
end_time = time.time()

# Print schedule
if sched is None:
    print("No solution.")
else:
    weeks = len(sched[0])
    print("        ", end="")
    for w in range(1, weeks+1): print(f"Week {w}".center(10), end="")
    print()
    for p, row in enumerate(sched, 1):
        print(f"Period {p}:", end=" ")
        for game in row:
            print(f"{game[0]} v {game[1]}".center(10), end="")
        print()

print(f"\nExecution time: {end_time - start_time:.2f} seconds")

#print for each team the imbalance (home - away)
for t in teams:
    home_games = sum(1 if is_true(m.evaluate(home[t, w, p])) else 0 for w in range(weeks) for p in range(periods))
    away_games = sum(1 if is_true(m.evaluate(away[t, w, p])) else 0 for w in range(weeks) for p in range(periods))
    imbalance = home_games - away_games
    print(f"Team {t+1}: Home games = {home_games}, Away games = {away_games}, Imbalance = {imbalance}")

          Week 1    Week 2    Week 3    Week 4    Week 5    Week 6    Week 7    Week 8    Week 9   Week 10   Week 11  
Period 1:   6 v 10    10 v 1    5 v 12    9 v 5     3 v 8     7 v 11    6 v 1     8 v 7     2 v 4     11 v 9    3 v 4   
Period 2:   11 v 8    9 v 8     7 v 10    2 v 3     1 v 5     2 v 6    11 v 12    5 v 4     9 v 6     12 v 4    1 v 7   
Period 3:   12 v 3    5 v 2     1 v 9     7 v 6    11 v 10    4 v 1     4 v 10    9 v 3     7 v 12    8 v 6     2 v 11  
Period 4:   5 v 7     3 v 11    6 v 3     1 v 11    4 v 6     12 v 9    9 v 2     10 v 2    8 v 1     5 v 10    12 v 8  
Period 5:   1 v 2     6 v 12    4 v 11    8 v 4     9 v 7     3 v 5     8 v 5     1 v 12    3 v 10    2 v 7     10 v 9  
Period 6:   4 v 9     4 v 7     2 v 8    10 v 12    12 v 2    10 v 8    7 v 3     6 v 11    11 v 5    3 v 1     5 v 6   

Execution time: 29.09 seconds
Team 1: Home games = 6, Away games = 5, Imbalance = 1
Team 2: Home games = 6, Away games = 5, Imbalance = 1
Team 3: Home gam